In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("DataSetForModel.csv")  # Replace with your CSV path
df = df[['text', 'label']].dropna()




In [2]:
# Text preprocessing
max_words = 5000
max_len = 150
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=max_len)# Labels
y = df['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:

# Build LSTM model
model = Sequential()
model.add(Embedding(max_words,64 ))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))  # For binary classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [4]:
# Train model
history = model.fit(X_train, y_train, epochs=4, batch_size=128, validation_split=0.2)

# Evaluate
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_classes))


Epoch 1/4
221/221 ━━━━━━━━━━━━━━━━━━━━ 64s 214ms/step - accuracy: 0.8488 - loss: 0.3695 - val_accuracy: 0.9789 - val_loss: 0.0690
Epoch 2/4
221/221 ━━━━━━━━━━━━━━━━━━━━ 44s 200ms/step - accuracy: 0.9808 - loss: 0.0644 - val_accuracy: 0.9845 - val_loss: 0.0512
Epoch 3/4
221/221 ━━━━━━━━━━━━━━━━━━━━ 43s 194ms/step - accuracy: 0.9890 - loss: 0.0394 - val_accuracy: 0.9818 - val_loss: 0.0728
Epoch 4/4
221/221 ━━━━━━━━━━━━━━━━━━━━ 35s 160ms/step - accuracy: 0.9815 - loss: 0.0590 - val_accuracy: 0.9852 - val_loss: 0.0495
276/276 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4536
           1       0.98      0.99      0.98      4274

    accuracy                           0.98      8810
   macro avg       0.98      0.98      0.98      8810
weighted avg       0.98      0.98      0.98      8810



In [6]:
model.save("my_model.keras")